In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import gurobipy as gp
import random
import time

from ModeloMatematico import ModeloMatematico

In [2]:
# myapp.py
import logging
logger = logging.getLogger(__name__)

logging.basicConfig(filename='myapp.log', level=logging.INFO, filemode="a+",
                        format="%(asctime)-15s %(levelname)-8s %(message)s")
logger.info('Started')


In [3]:
import pandas as pd
df = pd.read_excel("tabelas_2023-09-06.xlsx", "Tabela Distancia", usecols=["origin", "destination", "value"])
df_id = pd.read_excel("tabelas_2023-09-06.xlsx", "Pontos", usecols=["ID", "Demanda"])

In [4]:
Pontos_IDs = list(df_id["ID"].to_list())[1:]

In [5]:
INSTANCIAS = []#{"Pontos": [], "d_ij": [], "d_i": []}
i_s = [(0, 25),(25, 50), (50, 75), (75, 100), (100, 125), (125, 150), (150, 175), (175, 200)]
random.seed(1)
for i in range(len(i_s)):
    indices = list(range(i_s[i][0], i_s[i][1]))# list(random.choices(range(1, len(Pontos_IDs)+1), k=25))
    Pontos = [list(df_id["ID"].to_list())[0]] + [Pontos_IDs[i] for i in indices]
    d_i = [list(df_id["Demanda"].to_list())[i] for i in [0] + indices]
    
    dict_df = {(d["origin"], d["destination"]):d["value"]   for d in list(df.to_dict(orient="index").values())}
    d_ij = np.zeros(shape=(len(Pontos),len(Pontos)))

    for i, ID1 in enumerate(Pontos):
        for j, ID2 in enumerate(Pontos):
            if ID1==ID2: continue
            d_ij[i,j] = dict_df[(ID1, ID2)]  / 1000
    
    INSTANCIAS.append({"Pontos": Pontos, "d_ij": d_ij, "d_i": d_i})


In [7]:
# Caminho do arquivo Excel
caminho_arquivo = 'Resultados_V10.xlsx'

# Ler todas as abas
abas = pd.read_excel(caminho_arquivo, sheet_name=None)
dados = []
instancias = []
resumo = []
# Mostrar os nomes das abas e seus respectivos DataFrames
atualizar = True
if atualizar:
    for nome_aba, df in abas.items():
        print(f'Aba: {nome_aba}')
        #print(df.head())  # Exibe as primeiras linhas do DataFrame
        dado = df.to_dict(orient="records")
        print(dado)
        if nome_aba != "resumo":
            dados.append(dado)
            instancias.append(nome_aba)
        else:
            resumo = dado

Aba: Instancia_1
[{'variavel': 'x_ijk', 'i': 0, 'j': 7, 'k': 1, 'tipo': 1, 'value': 0.9999994348656724, 'obs': nan}, {'variavel': 'f_ijk', 'i': 0, 'j': 7, 'k': 1, 'tipo': 1, 'value': 5894.690000000001, 'obs': nan}, {'variavel': 'g_ijk', 'i': 0, 'j': 7, 'k': 1, 'tipo': 1, 'value': 0.0, 'obs': nan}, {'variavel': 'x_ijk', 'i': 4, 'j': 6, 'k': 1, 'tipo': 1, 'value': 0.9999994367564855, 'obs': nan}, {'variavel': 'f_ijk', 'i': 4, 'j': 6, 'k': 1, 'tipo': 1, 'value': 972.6494502969496, 'obs': nan}, {'variavel': 'g_ijk', 'i': 4, 'j': 6, 'k': 1, 'tipo': 1, 'value': 0.0, 'obs': nan}, {'variavel': 'x_ijk', 'i': 5, 'j': 4, 'k': 1, 'tipo': 1, 'value': 0.9999994357942653, 'obs': nan}, {'variavel': 'f_ijk', 'i': 5, 'j': 4, 'k': 1, 'tipo': 1, 'value': 1199.660969752703, 'obs': nan}, {'variavel': 'g_ijk', 'i': 5, 'j': 4, 'k': 1, 'tipo': 1, 'value': 0.0, 'obs': nan}, {'variavel': 'x_ijk', 'i': 6, 'j': 21, 'k': 1, 'tipo': 1, 'value': 0.9999994370778451, 'obs': nan}, {'variavel': 'f_ijk', 'i': 6, 'j': 21, 

In [8]:
# Algoritmo do vizinho mais próximo para o VRP
def vrp_vizinho_mais_proximo(pontos, num_veiculos, distancia, d_i):
    rotas = [[] for _ in range(num_veiculos)]  # Listas de rotas para cada veículo
    num_clientes = len(pontos)
    d_i = np.array(d_i)

    # Adiciona o ponto de partida (depósito) como o primeiro ponto
    deposito = pontos[0]
    clientes = pontos[1:].copy()  # Clientes são todos os pontos exceto o depósito
    
    for i in range(num_veiculos):
        rota = [deposito]  # Começa no depósito
        distancia_total = 0
        
        while clientes:
            ultimo_ponto = pontos[rota[-1]]
            # Encontra o cliente mais próximo
            
            proximo_cliente = min(clientes, key=lambda c: distancia[ultimo_ponto, c])
            #print(proximo_cliente)
            if distancia[ultimo_ponto, proximo_cliente] == np.inf:
                break
            elif d_i[rota+[proximo_cliente]].sum() > 7000:
                distancia[ultimo_ponto, proximo_cliente] = np.inf
            else:
                distancia_total += distancia[ultimo_ponto, proximo_cliente]
                clientes.remove(proximo_cliente)
                # Adiciona o índice do cliente na rota
                rota.append(pontos.index(proximo_cliente))

        rota.append(deposito)  # Retorna ao depósito
        rotas[i] = rota

    return rotas

In [9]:
# Heuristica:
from copy import copy
rotas_iniciais = []
for idx, instancia in enumerate(INSTANCIAS):
    Pontos = instancia["Pontos"]
    d_i = instancia["d_i"]
    d_ij = instancia["d_ij"]


    # SE QUANTIDADE DE ROTAS POR VEICULO IGUAL A 1 A FROTA E LIMITADA A |K|, CASO |K| = 1, PODE-SE REMOVER RESTRICAO (3) E TORNAR FROTA INFINITA.

    # CONJUNTOS
    N = list(range(len(Pontos))) # 1 CD e 10 CLientes

    Q = 7920 # CAPACIDADE LIQUIDA DO VEICULO (KG)

    

    rotas = vrp_vizinho_mais_proximo(N, 10, copy(d_ij), d_i)

    
    # VALIDAR ROTAS:
    
    d_i = np.array(d_i)
    rotas_dict = {"x_ijk": dict(), "f_ijk": dict(), "g_ijk": dict()}
    k = 0
    for rota in rotas:
        if len(rota)==2: continue
        if d_i[rota].sum() > Q:
            print("INVALIDO CARGA")
        bateria = 100
        for i_j in range(1, len(rota)):
            i = rota[i_j-1]
            j = rota[i_j]
            if i == 0:
                bateria -= d_ij[0, j] * 0.74
            elif j == 0:
                bateria -= d_ij[i, 0] * 0.57
            else:
                bateria -= d_ij[i, j] * 1.0

            rotas_dict["x_ijk"][(i,j,k,0)] = 1

            
        if bateria < 20:
            print("INVALIDO BATERIA")

        k+=1
    print(rotas_dict)



    #print(rotas_dict)
    

    rotas_iniciais.append(rotas_dict)

    

{'x_ijk': {(0, 21, 0, 0): 1, (21, 22, 0, 0): 1, (22, 6, 0, 0): 1, (6, 5, 0, 0): 1, (5, 4, 0, 0): 1, (4, 15, 0, 0): 1, (15, 8, 0, 0): 1, (8, 13, 0, 0): 1, (13, 14, 0, 0): 1, (14, 12, 0, 0): 1, (12, 11, 0, 0): 1, (11, 10, 0, 0): 1, (10, 9, 0, 0): 1, (9, 7, 0, 0): 1, (7, 16, 0, 0): 1, (16, 20, 0, 0): 1, (20, 19, 0, 0): 1, (19, 17, 0, 0): 1, (17, 18, 0, 0): 1, (18, 25, 0, 0): 1, (25, 24, 0, 0): 1, (24, 23, 0, 0): 1, (23, 1, 0, 0): 1, (1, 0, 0, 0): 1, (0, 2, 1, 0): 1, (2, 3, 1, 0): 1, (3, 0, 1, 0): 1}, 'f_ijk': {}, 'g_ijk': {}}
{'x_ijk': {(0, 12, 0, 0): 1, (12, 10, 0, 0): 1, (10, 11, 0, 0): 1, (11, 8, 0, 0): 1, (8, 9, 0, 0): 1, (9, 4, 0, 0): 1, (4, 5, 0, 0): 1, (5, 6, 0, 0): 1, (6, 7, 0, 0): 1, (7, 3, 0, 0): 1, (3, 23, 0, 0): 1, (23, 24, 0, 0): 1, (24, 21, 0, 0): 1, (21, 22, 0, 0): 1, (22, 20, 0, 0): 1, (20, 19, 0, 0): 1, (19, 18, 0, 0): 1, (18, 13, 0, 0): 1, (13, 14, 0, 0): 1, (14, 16, 0, 0): 1, (16, 2, 0, 0): 1, (2, 0, 0, 0): 1, (0, 1, 1, 0): 1, (1, 15, 1, 0): 1, (15, 17, 1, 0): 1, (17, 2

In [10]:
for idx, instancia in enumerate(INSTANCIAS):
    if f"Instancia_{idx+1}" in instancias: continue
    print(f"Instancia {idx+1}")

In [ ]:
#dados = []
#resumo = []
import time
for idx, instancia in enumerate(INSTANCIAS):
    if f"Instancia_{idx+1}" in instancias: continue
    t = time.time()
    Pontos = instancia["Pontos"]
    d_i = instancia["d_i"]
    d_ij = instancia["d_ij"]


    # SE QUANTIDADE DE ROTAS POR VEICULO IGUAL A 1 A FROTA E LIMITADA A |K|, CASO |K| = 1, PODE-SE REMOVER RESTRICAO (3) E TORNAR FROTA INFINITA.

    # CONJUNTOS
    N = list(range(len(Pontos))) # 1 CD e 10 CLientes
    C = N[1:] # CLIENTES
    K = list(range(10)) # 10 VEICULOS
    A = [(i,j) for i in N for j in N if i!=j] # ARCOS VALIDOS

    Q = 7920 # CAPACIDADE LIQUIDA DO VEICULO (KG)

    Cv_diesel = 2.3611 #$ / km
    cv_khw = 0.7258 / 0.89 #$ / kWh


    cf_eletrico =  24004.36 / 20  #$/dia
    cf_diessel =  17547.18 / 20  # $ / dia

    emissao_diesel = 3.2 / 4
    emissao_kwh = 0.0617
    #cf_diessel = cf_eletrico

    print(f"Instancia {idx+1}")
    logger.info(f'Resolvendo Instância {idx+1}')
    m, x_ijk, f_ijk, g_ijk, gap = ModeloMatematico(C, N, A, K, Q, d_i, d_ij, cf_diessel, Cv_diesel, cf_eletrico, cv_khw, rotainicial = rotas_iniciais[idx])

    
    resp = {tipo: {k: [] for k in K} for tipo in [0,1]}
    dados.append([])
    
    dist = 0
    emissao_eletrico = emissao_kwh * (sum(x_ijk[0,j,k, 0].X * d_ij[0, j] * 0.74 for j in C for k in K)
                                + sum(x_ijk[i,j,k, 0].X * d_ij[i, j] * 1.0 for i,j in A for k in K if i!=0 and j!=0)
                                + sum(x_ijk[i,0,k, 0].X * d_ij[i, 0] * 0.57 for i in C for k in K))
    emissao_diesel_Total   =  sum(emissao_diesel * d_ij[i,j]*x_ijk[i,j,k,1].X for i,j in A for k in K)                
    cust = m.objVal
    veiculos = [[],[]]
    for tipo in [0,1]:
        for k in K:
            for i,j in A:
                if x_ijk[i,j,k,tipo].X > 0.1: 
                    resp[tipo][k].append((i,j))
                    dados[-1].append({"variavel": "x_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": x_ijk[i,j,k,tipo].X, "obs": ""})
                    dados[-1].append({"variavel": "f_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": f_ijk[i,j,k,tipo].X, "obs": ""})
                    dados[-1].append({"variavel": "g_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": g_ijk[i,j,k].X, "obs": ""})
                    dist += d_ij[i,j]*x_ijk[i,j,k,tipo].X
                    
                    if k not in veiculos[tipo]: veiculos[tipo].append(k)

                else:
                    if f_ijk[i,j,k,tipo].X > 0.1 or g_ijk[i,j,k].X>0.1:
                        dados[-1].append({"variavel": "x_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": x_ijk[i,j,k,tipo].X, "obs": "Sim"})
                        dados[-1].append({"variavel": "f_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": f_ijk[i,j,k,tipo].X, "obs": "Sim"})
                        dados[-1].append({"variavel": "g_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": g_ijk[i,j,k].X, "obs": "Sim"})

    resumo.append({"Instancia": idx, "dist": dist, "emissao_eletrico": emissao_eletrico, "emissao_diesel": emissao_diesel_Total, "cust": cust, "qnt_v_eletricos": len(veiculos[0]), "qnt_v_diesel": len(veiculos[1]), "tempo": time.time() - t, "gap": gap})
    t = time.time()
    for tipo in [0,1]:
        for k in K:
            if not len(resp[tipo][k]): continue
            print(resp[tipo][k])
            logger.info(f'Rota {tipo} {resp[tipo][k]}')

    with pd.ExcelWriter(f"./Resultados_V10.xlsx") as writer:
        for i, dado in enumerate(dados):
            df = pd.DataFrame(dado)
            df.to_excel(writer, sheet_name=f'Instancia_{i+1}', index=False)
        df = pd.DataFrame(resumo)
        df.to_excel(writer, sheet_name=f'resumo', index=False)
    


In [11]:
import pandas as pd
with pd.ExcelWriter(f"./Resultados_V10.xlsx") as writer:
    for i, dado in enumerate(dados):
        df = pd.DataFrame(dado)
        df.to_excel(writer, sheet_name=f'Instancia_{i+1}', index=False)
    df = pd.DataFrame(resumo)
    df.to_excel(writer, sheet_name=f'resumo', index=False)
    

In [ ]:
for idx, instancia in enumerate(INSTANCIAS):
    Pontos = instancia["Pontos"]
    d_i = instancia["d_i"]
    d_ij = instancia["d_ij"]


    # SE QUANTIDADE DE ROTAS POR VEICULO IGUAL A 1 A FROTA E LIMITADA A |K|, CASO |K| = 1, PODE-SE REMOVER RESTRICAO (3) E TORNAR FROTA INFINITA.

    # CONJUNTOS
    N = list(range(len(Pontos))) # 1 CD e 10 CLientes
    C = N[1:] # CLIENTES
    K = list(range(10)) # 10 VEICULOS
    A = [(i,j) for i in N for j in N if i!=j] # ARCOS VALIDOS

    Q = 7920 # CAPACIDADE LIQUIDA DO VEICULO (KG)

    Cv_diesel = 2.3611 #$ / km
    cv_khw = 0.7258 / 0.89 #$ / kWh


    cf_eletrico =  24004.36 / 20  #$/dia
    cf_diessel =  17547.18 / 20  # $ / dia

    emissao_diesel = 3.2 / 4
    emissao_kwh = 0.0617

    emissoes_cenariosBase = [55.6411110305023, 59.99180281, 78.00465719, 124.2078885, 70.46351267, 68.09769779, 80.96092796, 82.50884843]
 

    #emissao_atual = emissoes_cenariosBase[idx]["emissao_eletrico"]+resumo[idx]["emissao_diesel"]
    emissao_atual = emissoes_cenariosBase[idx]
    #cf_diessel = cf_eletrico
    for percent_reduct in [0.4, 0.6, 0.8]:
        t = time.time()
        print(f"Instancia {idx+1} {percent_reduct}")
        logger.info(f'Resolvendo Instância {idx+1} {percent_reduct}')
        m, x_ijk, f_ijk, g_ijk, gap = ModeloMatematico(C, N, A, K, Q, d_i, d_ij, cf_diessel, Cv_diesel, cf_eletrico, cv_khw, rodarEmissao = True, LimiteEmissao = percent_reduct * emissao_atual, rotainicial = rotas_iniciais[idx])

        
        resp = {tipo: {k: [] for k in K} for tipo in [0,1]}
        dados.append([])
        
        dist = 0
        emissao_eletrico = emissao_kwh * (sum(1 if x_ijk[0,j,k, 0].X >0.1 else 0 * d_ij[0, j] * 0.74 for j in C for k in K)
                                    + sum(1 if x_ijk[i,j,k, 0].X>0.1 else 0 * d_ij[i, j] * 1.0 for i,j in A for k in K if i!=0 and j!=0)
                                    + sum(1 if x_ijk[i,0,k, 0].X else 0 * d_ij[i, 0] * 0.57 for i in C for k in K))
        emissao_diesel_Total   = emissao_diesel *  sum(d_ij[i,j]*1 if x_ijk[i,j,k,1].X>0.1 else 0 for i,j in A for k in K)                
        cust = m.objVal
        veiculos = [[],[]]
        for tipo in [0,1]:
            for k in K:
                for i,j in A:
                    if x_ijk[i,j,k,tipo].X > 0.1: 
                        resp[tipo][k].append((i,j))
                        dados[-1].append({"variavel": "x_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": x_ijk[i,j,k,tipo].X, "obs": ""})
                        dados[-1].append({"variavel": "f_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": f_ijk[i,j,k,tipo].X, "obs": ""})
                        dados[-1].append({"variavel": "g_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": g_ijk[i,j,k].X, "obs": ""})
                        dist += d_ij[i,j]*x_ijk[i,j,k,tipo].X
                        
                        if k not in veiculos[tipo]: veiculos[tipo].append(k)

                    else:
                        if f_ijk[i,j,k,tipo].X > 0.1 or g_ijk[i,j,k].X>0.1:
                            dados[-1].append({"variavel": "x_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": x_ijk[i,j,k,tipo].X, "obs": "Sim"})
                            dados[-1].append({"variavel": "f_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": f_ijk[i,j,k,tipo].X, "obs": "Sim"})
                            dados[-1].append({"variavel": "g_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": g_ijk[i,j,k].X, "obs": "Sim"})
        
        resumo[idx][f"percent_reduct_{percent_reduct}"] = percent_reduct
        resumo[idx][f"dist_{percent_reduct}"] = dist
        resumo[idx][f"emissao_eletrico_{percent_reduct}"] = emissao_eletrico
        resumo[idx][f"emissao_diesel_{percent_reduct}"] = emissao_diesel_Total
        resumo[idx][f"cust_{percent_reduct}"] = cust
        resumo[idx][f"qnt_v_eletricos_{percent_reduct}"] = len(veiculos[0])
        resumo[idx][f"qnt_v_diesel_{percent_reduct}"] = len(veiculos[1])
        resumo[idx][f"gap{percent_reduct}"] = gap
        resumo[idx][f"time_{percent_reduct}"] = time.time() - t
        

        t = time.time()

        for tipo in [0,1]:
            for k in K:
                if not len(resp[tipo][k]): continue
                print(resp[tipo][k])
                logger.info(f'Rota {tipo} {resp[tipo][k]}')

    with pd.ExcelWriter(f"./Resultados_V11.xlsx") as writer:
        for i, dado in enumerate(dados):
            df = pd.DataFrame(dado)
            df.to_excel(writer, sheet_name=f'Instancia_{i+1}')
        df = pd.DataFrame(resumo)
        df.to_excel(writer, sheet_name=f'resumo')

In [ ]:
'''for idx, instancia in enumerate(INSTANCIAS):
    Pontos = instancia["Pontos"]
    d_i = instancia["d_i"]
    d_ij = instancia["d_ij"]


    # SE QUANTIDADE DE ROTAS POR VEICULO IGUAL A 1 A FROTA E LIMITADA A |K|, CASO |K| = 1, PODE-SE REMOVER RESTRICAO (3) E TORNAR FROTA INFINITA.

    # CONJUNTOS
    N = list(range(len(Pontos))) # 1 CD e 10 CLientes
    C = N[1:] # CLIENTES
    K = list(range(10)) # 10 VEICULOS
    A = [(i,j) for i in N for j in N if i!=j] # ARCOS VALIDOS

    Q = 7000 # CAPACIDADE LIQUIDA DO VEICULO (KG)

    Cv_diesel = 1.17 #$ / km
    cv_khw = 0.4 #$ / kWh


    cf_eletrico = 742 #$/dia
    cf_diessel = 443 # $ / dia

    emissao_diesel = 3.2 / 4
    emissao_kwh = 0.0617


    emissao_atual = resumo[idx]["emissao_eletrico"]+resumo[idx]["emissao_diesel"]
    #cf_diessel = cf_eletrico
    for percent_reduct in [0.4, 0.6, 0.8]:
        t = time.time()
        print(f"Instancia {idx+1} {percent_reduct}")
        logger.info(f'Resolvendo Instância {idx+1} {percent_reduct}')
        m, x_ijk, f_ijk, g_ijk = ModeloMatematico(C, N, A, K, Q, d_i, d_ij, cf_diessel, Cv_diesel, cf_eletrico, cv_khw, rodarEmissao = True, LimiteEmissao = percent_reduct * emissao_atual)

        
        resp = {tipo: {k: [] for k in K} for tipo in [0,1]}
        dados.append([])
        
        dist = 0
        emissao_eletrico = emissao_kwh * (sum(x_ijk[0,j,k, 0].X * d_ij[0, j] * 0.74 for j in C for k in K)
                                    + sum(x_ijk[i,j,k, 0].X * d_ij[i, j] * 1.0 for i,j in A for k in K if i!=0 and j!=0)
                                    + sum(x_ijk[i,0,k, 0].X * d_ij[i, 0] * 0.57 for i in C for k in K))
        emissao_diesel   =  sum(emissao_diesel * d_ij[i,j]*x_ijk[i,j,k,1].X for i,j in A for k in K)                
        cust = m.objVal
        veiculos = [[],[]]
        for tipo in [0,1]:
            for k in K:
                for i,j in A:
                    if x_ijk[i,j,k,tipo].X > 0.1: 
                        resp[tipo][k].append((i,j))
                        dados[-1].append({"variavel": "x_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": x_ijk[i,j,k,tipo].X, "obs": ""})
                        dados[-1].append({"variavel": "f_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": f_ijk[i,j,k,tipo].X, "obs": ""})
                        dados[-1].append({"variavel": "g_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": g_ijk[i,j,k].X, "obs": ""})
                        dist += d_ij[i,j]*x_ijk[i,j,k,tipo].X
                        
                        if k not in veiculos[tipo]: veiculos[tipo].append(k)

                    else:
                        if f_ijk[i,j,k,tipo].X > 0.1 or g_ijk[i,j,k].X>0.1:
                            dados[-1].append({"variavel": "x_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": x_ijk[i,j,k,tipo].X, "obs": "Sim"})
                            dados[-1].append({"variavel": "f_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": f_ijk[i,j,k,tipo].X, "obs": "Sim"})
                            dados[-1].append({"variavel": "g_ijk", "i": i, "j": j, "k": k, "tipo": tipo, "value": g_ijk[i,j,k].X, "obs": "Sim"})
        
        resumo[idx][f"percent_reduct_{percent_reduct}"] = percent_reduct
        resumo[idx][f"dist_{percent_reduct}"] = dist
        resumo[idx][f"emissao_eletrico_{percent_reduct}"] = emissao_eletrico
        resumo[idx][f"emissao_diesel_{percent_reduct}"] = emissao_diesel
        resumo[idx][f"cust_{percent_reduct}"] = cust
        resumo[idx][f"qnt_v_eletricos_{percent_reduct}"] = len(veiculos[0])
        resumo[idx][f"qnt_v_diesel_{percent_reduct}"] = len(veiculos[1])
        resumo[idx][f"time_{percent_reduct}"] = time.time() - t

        t = time.time()

        for tipo in [0,1]:
            for k in K:
                if not len(resp[tipo][k]): continue
                print(resp[tipo][k])
                logger.info(f'Rota {tipo} {resp[tipo][k]}')

    with pd.ExcelWriter(f"./Resultados.xlsx") as writer:
        for i, dado in enumerate(dados):
            df = pd.DataFrame(dado)
            df.to_excel(writer, sheet_name=f'Instancia_{i+1}')
        df = pd.DataFrame(resumo)
        df.to_excel(writer, sheet_name=f'resumo')'''

In [15]:
logger.info('Finished')